In [13]:
from datasets import load_dataset

# Base URL to your hosted parquet files on Hugging Face
base_url = "https://huggingface.co/datasets/ChernovAndrei/reco-fm-data/resolve/main"

# List of parquet file names to load
files = {
    "train": "train_data.parquet",
    "val": "val_data.parquet",
    "test": "test_data.parquet",
    "title_embeddings": "title_embeddings.parquet"
}

# Load each dataset individually
datasets = {}
for split_name, filename in files.items():
    print(f"Loading {split_name} from {filename}...")
    datasets[split_name] = load_dataset(
        path="parquet",
        data_files=f"{base_url}/{filename}",
        split=None
    )
    # print(f"{split_name} loaded. Rows: {len(datasets[split_name])}")



Loading train from train_data.parquet...
Loading val from val_data.parquet...
Loading test from test_data.parquet...
Loading title_embeddings from title_embeddings.parquet...


In [18]:
# Example usage:
print("\nExample train sample:")
datasets["train"]['train'].to_pandas().head()


Example train sample:


,user_id,parent_asin,rating,timestamp,history
0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07J3GH1W1,5.0,1547589356557,
1,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07W397QG4,5.0,1593352422858,B07J3GH1W1
2,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07KG1TWP5,5.0,1596473351088,B07J3GH1W1 B07W397QG4
3,AFSKPY37N3C43SOI5IEXEK5JSIYA,B08JTNQFZY,5.0,1617904219785,B07J3GH1W1 B07W397QG4 B07KG1TWP5
4,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07SLFWZKN,3.0,1619737501209,B07J3GH1W1 B07W397QG4 B07KG1TWP5 B08JTNQFZY


In [19]:
datasets["test"]['train'].to_pandas().head()

,user_id,parent_asin,rating,timestamp,history
0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B08P2DZB4X,5.0,1627391044559,B07J3GH1W1 B07W397QG4 B07KG1TWP5 B08JTNQFZY B0...
1,AHV6QCNBJNSGLATP56JAWJ3C4G2A,B0BTJ6SYKB,4.0,1623188037011,B07NPWK167 B07SW7D6ZR B07WNBZQGT B082NKQ4ZT B0...
2,AFJBKPK5W56XWSNPQU2WW66ISWYQ,B08XJWLLKQ,5.0,1618934798418,B07J3GH1W1 B07LCHCD6Q B084ZHP45Y B087Z9X39L B0...
3,AFXF3EGQTQDXMRLDWFU7UBFQZB7Q,B08HVRP54L,3.0,1653607216628,B07KXM94BT B07LCHCD6Q B0837K9W6P B081ZN3TD5 B0...
4,AFWVN52MRBWOTIK7UGXBWGOY4HBA,B08SBV6Z57,5.0,1625801142291,B07LCHCD6Q B086VYKNDF B07W397QG4 B08879N2W3 B0...
